# Vertical scaling of columnar analysis: Numba and C++ integration

(no-frills notebook)

<br><br><br><br><br>

## Motivation

There are two kinds of languages, and Python is one of the slow kind.

<img src="img/benchmark-games-2023.svg" width="100%">

<br><br><br><br><br>

But there are a lot of ways to connect Python with compiled languages, usually C++ (increasingly, Rust).

<img src="img/history-of-bindings-2.svg" width="100%">

<br><br><br><br><br>

## Setting up an example problem

First, let's get a biggish dataset...

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import awkward as ak
import vector
vector.register_awkward()

In [ ]:
events = ak.from_parquet("data/SMHiggsToZZTo4L.parquet")

In [ ]:
events.muon

...and then let's give it some work to do. How about dimuon pairs? (The "hello world" of HEP.)

This uses the columnar methods from last week (CoDaS-HEP).

In [ ]:
muplus = events.muon[events.muon.charge > 0]
muminus = events.muon[events.muon.charge < 0]

In [ ]:
mu1, mu2 = ak.unzip(ak.cartesian((muplus, muminus)))

In [ ]:
plt.hist(ak.ravel(
    
    (mu1 + mu2).mass

), bins=100, range=(0, 100));

It returns the right answer, but maybe we don't like the performance. The sum

```python
(mu1 + mu2)
```

converts cylindrical coordinates into Cartesian, for all four coordinates, and then

```python
(mu1 + mu2).mass
```

computes $m = \sqrt{E^2 - {p_x}^2 - {p_y}^2 - {p_z}^2}$ by computing arrays for each intermediate step. We can do better than that by

- not computing unnecessary coordinates
- not creating and filling unnecessary arrays.

We can do the first part with another formula (which ignores the mass of the muon):

$$m \approx \sqrt{2 \, {p_T}_1 \, {p_T}_2 \, \left(\cosh\left({\eta}_1 - {\eta}_2\right) - \cos\left({\phi}_1 - {\phi}_2\right) \right)}$$

In [ ]:
def fast_mass(mu1, mu2):
    return np.sqrt(
        2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
    )

In [ ]:
plt.hist(ak.ravel(
    
    fast_mass(mu1, mu2)

), bins=100, range=(0, 100));

In [ ]:
def fast_mass_pedantic(mu1, mu2):
    tmp1 = mu1.eta - mu2.eta
    tmp2 = np.cosh(tmp1)
    del tmp1
    tmp3 = mu1.phi - mu2.phi
    tmp4 = np.cos(tmp3)
    del tmp3
    tmp5 = tmp2 - tmp4
    del tmp2
    del tmp4
    tmp6 = mu2.pt * tmp5
    del tmp5
    tmp7 = mu1.pt * tmp6
    del tmp6
    tmp8 = 2 * tmp7
    del tmp7
    tmp9 = np.sqrt(tmp8)
    del tmp8
    return tmp9

In [ ]:
plt.hist(ak.ravel(
    
    fast_mass_pedantic(mu1, mu2)

), bins=100, range=(0, 100));

In [ ]:
%%timeit

fast_mass(mu1, mu2)

In [ ]:
%%timeit

fast_mass_pedantic(mu1, mu2)

There are lots of reasons why allocating new arrays and copying results to them for each _step_ in a formula is slower than iterating over the input and computing the _whole_ formula before moving on to the next input.

- Allocating memory is expensive (a _search_ through fragmented RAM).
- Accessing new areas of RAM is expensive because it bypasses the CPU caching mechanism.
- Most mathematical calculations are faster than fetching data from RAM (into CPU caches).

<br><br><br><br><br>

## The example problem in C++

See [this minicourse](https://github.com/henryiii/python-compiled-minicourse) (from [iscinumpy.dev](https://iscinumpy.dev/)) for simple ways and robust ways to wrap C++ code.

This is a simple way: [pybind11](https://pybind11.readthedocs.io/) is good, but if we want this to compile everywhere (including Windows), we should build makefiles with [CMake](https://cliutils.gitlab.io/modern-cmake/) and [scikit-build](https://scikit-build.readthedocs.io/).

In [ ]:
%%writefile test1_pybind11.cpp

#include <pybind11/pybind11.h>
namespace py = pybind11;

float run(float mu1pt, float mu2pt, float mu1eta, float mu2eta, float mu1phi, float mu2phi) {
    return sqrt(
        2 * mu1pt * mu2pt * (cosh(mu1eta - mu2eta) - cos(mu1phi - mu2phi))
    );
}

PYBIND11_MODULE(test1_pybind11, m) {
    m.def("run", &run);
}

The next two cells find compiler arguments (in Python) and then run the compiler (in the shell).

In [ ]:
import os
import sys
from pybind11 import get_include

inc = "-I " + get_include()
plat = "-undefined dynamic_lookup" if "darwin" in sys.platform else "-fPIC"
pyinc = !python3-config --cflags

In [ ]:
!c++ -std=c++11 test1_pybind11.cpp -shared {inc} {pyinc.s} -o test1_pybind11.so {plat}

And the next cell loads the library and runs it.

In [ ]:
import test1_pybind11

test1_pybind11.run(mu1.pt[0, 0], mu2.pt[0, 0], mu1.eta[0, 0], mu2.eta[0, 0], mu1.phi[0, 0], mu2.phi[0, 0])

In [ ]:
results = []
for i in range(len(mu1)):
    for j in range(len(mu1[i])):
        results.append(
            test1_pybind11.run(mu1.pt[i, j], mu2.pt[i, j], mu1.eta[i, j], mu2.eta[i, j], mu1.phi[i, j], mu2.phi[i, j])
        )

In [ ]:
plt.hist(results, bins=100, range=(0, 100));

That took much too long! I'm not even going to measure it.

Why? **(Don't scroll down!)**

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

Instead of calling the C++ function in a Python loop, let's put the loop in C++.

In [ ]:
%%writefile test2_pybind11.cpp

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
namespace py = pybind11;

void run(
    py::array_t<float> results,
    py::array_t<float, py::array::c_style | py::array::forcecast> mu1pt,
    py::array_t<float, py::array::c_style | py::array::forcecast> mu2pt,
    py::array_t<float, py::array::c_style | py::array::forcecast> mu1eta,
    py::array_t<float, py::array::c_style | py::array::forcecast> mu2eta,
    py::array_t<float, py::array::c_style | py::array::forcecast> mu1phi,
    py::array_t<float, py::array::c_style | py::array::forcecast> mu2phi
) {
    float* results_ = results.mutable_data();
    const float* mu1pt_ = mu1pt.data();
    const float* mu2pt_ = mu2pt.data();
    const float* mu1eta_ = mu1eta.data();
    const float* mu2eta_ = mu2eta.data();
    const float* mu1phi_ = mu1phi.data();
    const float* mu2phi_ = mu2phi.data();

    for (int i = 0;  i < results.size();  i++) {
        results_[i] = sqrt(
            2 * mu1pt_[i] * mu2pt_[i] * (cosh(mu1eta_[i] - mu2eta_[i]) - cos(mu1phi_[i] - mu2phi_[i]))
        );
    }
}

PYBIND11_MODULE(test2_pybind11, m) {
    m.def("run", &run);
}

In [ ]:
!c++ -std=c++11 test2_pybind11.cpp -shared {inc} {pyinc.s} -o test2_pybind11.so {plat}

In [ ]:
import test2_pybind11

In [ ]:
mu1pt = ak.to_numpy(ak.ravel(mu1.pt))
mu2pt = ak.to_numpy(ak.ravel(mu2.pt))
mu1eta = ak.to_numpy(ak.ravel(mu1.eta))
mu2eta = ak.to_numpy(ak.ravel(mu2.eta))
mu1phi = ak.to_numpy(ak.ravel(mu1.phi))
mu2phi = ak.to_numpy(ak.ravel(mu2.phi))
assert len(mu1pt) == len(mu2pt) == len(mu1eta) == len(mu2eta) == len(mu1phi) == len(mu2phi)

results = np.empty(len(mu1pt), dtype=np.float32)
test2_pybind11.run(results, mu1pt, mu2pt, mu1eta, mu2eta, mu1phi, mu2phi)

In [ ]:
plt.hist(results, bins=100, range=(0, 100));

Now _that_ was fast.

In [ ]:
%%timeit

(mu1 + mu2).mass

In [ ]:
%%timeit

results = np.empty(len(mu1pt), dtype=np.float32)
test2_pybind11.run(results, mu1pt, mu2pt, mu1eta, mu2eta, mu1phi, mu2phi)

It was fast, but required a lot of work.

It's the best way to wrap C++ code as a library, but if you want to interleave snippets of C++ in a mostly Python script/notebook, you can use

- [ROOT](https://root.cern/)'s [gInterpreter.Declare](https://root.cern.ch/doc/master/classTInterpreter.html#a84a4890123faea8e740cd9f8d690e1e3)
- [ROOT](https://root.cern/)'s [RDataFrame](https://root.cern/doc/master/classROOT_1_1RDataFrame.html)
- [cppyy](https://cppyy.readthedocs.io/) (see [demo](https://github.com/henryiii/python-compiled-minicourse/blob/master/06-cppyy/06-cppyy.ipynb))

Any method that integrates Python and C++ has to translate data structures between the two languages. C++ has no Awkward Array, but NumPy arrays can be interpreted as `std::array` of numeric types.

The problem of integrating Python and C++ often comes down to deconstructing Python objects into NumPy arrays, passing them over, and building NumPy output back into Python objects.

<br><br><br><br><br>

## The example problem in Numba

In [ ]:
import numba as nb

Also for interleaving compiled code in a mostly Python script/notebook, Numba does two things differently:

- it uses Python syntax instead of C++ syntax
- it translates objects for you/presents an illusion that you're using Python objects throughout.

In [ ]:
def this_is_slow():
    out = 0.0
    for i in range(1000000):
        out += i
        out *= 0.5
    return out

this_is_slow()

In [ ]:
%%timeit

this_is_slow()

In [ ]:
@nb.njit
def this_is_fast():
    out = 0.0
    for i in range(1000000):
        out += i
        out *= 0.5
    return out

this_is_fast()

In [ ]:
%%timeit

this_is_fast()

The `@nb.njit` (no-Python Just-In-Time compilation) looks like magic, but there's a catch:

- not all Python code can be accelerated
- when it can't be accelerated, the error message is often formidable

In [ ]:
@nb.njit
def this_is_not_working():
    out = "bad"
    for i in range(1000000):
        if i == 0:
            out = 0.0
        out += i
        out *= 0.5
    return out

this_is_not_working()

In [ ]:
class SomethingIMadeUp:
    pass

something_I_made_up = SomethingIMadeUp()

@nb.njit
def this_is_not_working_either(x):
    return x

this_is_not_working_either(something_I_made_up)

Numba only knows about [these Python features](https://numba.readthedocs.io/en/stable/reference/pysupported.html) and [these NumPy functions](https://numba.readthedocs.io/en/stable/reference/numpysupported.html).

And any that someone has written an explicit extension for.

<br><br><br><br><br>

Awkward Array has been extended to Numba.

None of the `ak.*` functions or fancy slices are supported, but basic iteration is.

In [ ]:
@nb.njit
def mass_in_numba(results, mu1_everything, mu2_everything):
    k = 0
    for mu1_event, mu2_event in zip(mu1_everything, mu2_everything):
        for m1, m2 in zip(mu1_event, mu2_event):
            results[k] = np.sqrt(
                2 * m1.pt * m2.pt * (np.cosh(m1.eta - m2.eta) - np.cos(m1.phi - m2.phi))
            )
            k += 1

results = np.zeros(len(mu1pt), dtype=np.float32)
mass_in_numba(results, mu1, mu2)

In [ ]:
plt.hist(results, bins=100, range=(0, 100));

Compare:

In [ ]:
%%timeit

(mu1 + mu2).mass

In [ ]:
%%timeit

results = np.empty(len(mu1pt), dtype=np.float32)
test2_pybind11.run(results, mu1pt, mu2pt, mu1eta, mu2eta, mu1phi, mu2phi)

In [ ]:
%%timeit

results = np.empty(len(mu1pt), dtype=np.float32)
mass_in_numba(results, mu1, mu2)

<br><br><br><br><br>

### Numba vectorize

Numba has several ways to compile code. (https://numba.readthedocs.io/en/stable/user/vectorize.html) makes a function in which `output[i] = f(input[i])` for a given `f`.

(The signature in `@nb.vectorize(...)` will not be required for long: [numba/numba#8995](https://github.com/numba/numba/pull/8995).)

In [ ]:
@nb.vectorize([nb.float32(nb.float32, nb.float32, nb.float32, nb.float32, nb.float32, nb.float32)])
def mass_in_numba_vectorize(mu1pt, mu2pt, mu1eta, mu2eta, mu1phi, mu2phi):
    return np.sqrt(
        2 * mu1pt * mu2pt * (np.cosh(mu1eta - mu2eta) - np.cos(mu1phi - mu2phi))
    )

mass_in_numba_vectorize(mu1.pt, mu2.pt, mu1.eta, mu2.eta, mu1.phi, mu2.phi)

In [ ]:
%%timeit

results = mass_in_numba_vectorize(mu1.pt, mu2.pt, mu1.eta, mu2.eta, mu1.phi, mu2.phi)

<br><br><br><br><br>

### Creating an Awkward Array in a Numba-compiled function

[ArrayBuilder](https://awkward-array.org/doc/main/reference/generated/ak.ArrayBuilder.html) makes Awkward Arrays, discovering the type from the order in which methods are called. It can also be used in Numba-compiled functions.

- `ArrayBuilder` must be constructed outside the Numba-compiled function.
- Its `snapshot` method must be called outside the Numba-compiled function.

In [ ]:
@nb.njit
def mass_in_numba_awkward_output(builder, mu1_everything, mu2_everything):
    for mu1_event, mu2_event in zip(mu1_everything, mu2_everything):
        builder.begin_list()
        for m1, m2 in zip(mu1_event, mu2_event):
            builder.append(
                np.sqrt(
                    2 * m1.pt * m2.pt * (np.cosh(m1.eta - m2.eta) - np.cos(m1.phi - m2.phi))
                )
            )
        builder.end_list()

builder = ak.ArrayBuilder()
mass_in_numba_awkward_output(builder, mu1, mu2)
builder.snapshot()

Dynamically discovering types is a bit slow (even though it's implemented in C++).

In [ ]:
%%timeit

builder = ak.ArrayBuilder()
mass_in_numba_awkward_output(builder, mu1, mu2)
builder.snapshot()

<br><br><br><br><br>

### Using the new LayoutBuilder

`LayoutBuilder` is a brand-new (but lower-level) interface that lets you specify the data type upfront, gaining back some speed.

(There's also a [LayoutBuilder in C++](https://github.com/scikit-hep/awkward/blob/main/header-only/layout-builder/awkward/LayoutBuilder.h) for building Awkward Arrays from C++.)

In [ ]:
import awkward.numba.layoutbuilder as lb

In [ ]:
@nb.njit
def mass_in_numba_awkward_output2(builder, mu1_everything, mu2_everything):
    for mu1_event, mu2_event in zip(mu1_everything, mu2_everything):
        subbuilder = builder.begin_list()
        for m1, m2 in zip(mu1_event, mu2_event):
            subbuilder.append(
                np.sqrt(
                    2 * m1.pt * m2.pt * (np.cosh(m1.eta - m2.eta) - np.cos(m1.phi - m2.phi))
                )
            )
        builder.end_list()

builder = lb.ListOffset(np.int64, lb.Numpy(np.float32))
mass_in_numba_awkward_output2(builder, mu1, mu2)
ak.Array(builder.snapshot())

In [ ]:
%%timeit

builder = lb.ListOffset(np.int64, lb.Numpy(np.float32))
mass_in_numba_awkward_output2(builder, mu1, mu2)
ak.Array(builder.snapshot())

<br><br><br><br><br>

### Using Vector in Numba-compiled functions

The [Vector](https://vector.readthedocs.io/) library is also a Numba extension.

In [ ]:
@nb.njit
def mass_in_numba_with_vector(results, mu1_everything, mu2_everything):
    k = 0
    for mu1_event, mu2_event in zip(mu1_everything, mu2_everything):
        for m1, m2 in zip(mu1_event, mu2_event):
            results[k] = (m1 + m2).mass
            k += 1

results = np.zeros(len(mu1pt), dtype=np.float32)
mass_in_numba_with_vector(results, mu1, mu2)

In [ ]:
plt.hist(results, bins=100, range=(0, 100));

In [ ]:
%%timeit

results = np.empty(len(mu1pt), dtype=np.float32)
mass_in_numba_with_vector(results, mu1, mu2)

But now we're back to converting the vectors to Cartesian coordinates and calculating $m = \sqrt{E^2 - {p_x}^2 - {p_y}^2 - {p_z}^2}$ (though it's compiled and optimized by LLVM).

However, compare to Vector without Numba:

In [ ]:
%%timeit

(mu1 + mu2).mass

<br><br><br><br><br>

### Filling histograms (hist) in Numba-compiled functions

Coming ~~soon~~ someday...